In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dropout,Dense,Embedding,LSTM,Bidirectional
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report

In [14]:
news_train=fetch_20newsgroups(subset='train')
news_test=fetch_20newsgroups(subset='test')

In [15]:
len(news_train)

print(len(news_train.data))

print(news_train.target_names)
label_names=news_train.target_names

11314
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [16]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(news_train.data)
tokenizer.fit_on_texts(news_test.data)

X_train=tokenizer.texts_to_sequences(news_train.data)
X_train=pad_sequences(X_train,maxlen=400)
Y_train=to_categorical(news_train.target)
print(Y_train[0])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [17]:
X_test=tokenizer.texts_to_sequences(news_test.data)
X_test=pad_sequences(X_test,maxlen=400)
Y_test=to_categorical(news_test.target)
print(Y_test[0])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [28]:
voc_size  =   len(tokenizer.word_index) +1

model=Sequential()
model.add(Embedding(voc_size,128))
model.add(Bidirectional(LSTM(128,dropout=0.2,recurrent_dropout=0.15,return_sequences=True)))
model.add(Bidirectional(LSTM(64,dropout=0.2,recurrent_dropout=0.15,return_sequences=False)))
model.add(Dense(len(news_train.target_names),activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

#model=keras.models.load_model("sentiment_model")

print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 128)         22938880  
                                                                 
 bidirectional_12 (Bidirecti  (None, None, 256)        263168    
 onal)                                                           
                                                                 
 bidirectional_13 (Bidirecti  (None, 128)              164352    
 onal)                                                           
                                                                 
 dense_7 (Dense)             (None, 20)                2580      
                                                                 
Total params: 23,368,980
Trainable params: 23,368,980
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X_train,Y_train,batch_size=64,epochs=6)



Epoch 1/6


In [ ]:
#model.evaluate(X_test,Y_test)
model.save('sentiment_model')

In [ ]:
Y_pred=model.predict(X_train)
#Y_pred=model.predict(X_test)


In [ ]:
DEFAULT_CLASS=0
Y_pred2 = [np.argmax(x) if np.max(x) >= 0.5 else DEFAULT_CLASS for x in Y_pred]

#Y_pred2=np.argmax(Y_pred, axis=1)
#Y_test2=np.argmax(Y_test, axis=1)
Y_train2=np.argmax(Y_train,axis=1)
cm = confusion_matrix(Y_train2, Y_pred2)
#cm = confusion_matrix(Y_test2, Y_pred2)

sns.heatmap(cm)
plt.show()

print(classification_report(Y_train2,Y_pred2,target_names=label_names))
#print(classification_report(Y_test2,Y_pred2,target_names=label_names))

In [ ]:
Y_pred=model.predict(X_test)

DEFAULT_CLASS=0
Y_pred2 = [np.argmax(x) if np.max(x) >= 0.5 else DEFAULT_CLASS for x in Y_pred]

Y_test2=np.argmax(Y_test, axis=1)
cm = confusion_matrix(Y_test2, Y_pred2)

sns.heatmap(cm)
plt.show()

print(classification_report(Y_test2,Y_pred2,target_names=label_names))
#print(classification_report(Y_test2,Y_pred2,target_names=label_names))